In [1]:
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

In [2]:
structures_path = Path("../structures/irc_tests/irc_new_ts/")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

2025-07-15 17:00:40 INFO  frust.stepper: Working dir: .


In [3]:
def run_calc(df, debug=False, n_cores=12, mem_gb=30):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path("results_ts34-2")
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="IRC",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)
    
    options = {
        "wB97X-D3" : None,
        "6-31+G**"  : None,
        "TightSCF" : None,
        "Freq"     : None,
        "NoSym"    : None,
    }

    df1 = step.orca(df, "DFT", options)
    df1.to_parquet(f"{results_dir}/results_{name}.parquet")

In [4]:
import submitit

DEBUG           = False
N_CORES         = 8
MEM_GB          = 30
TIMEOUT_MIN     = 14400


executor = submitit.AutoExecutor("logs/ts34-2")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

for dfi in dfs:
    name = "ts34-" + dfi["custom_name"].iloc[0] 
    executor.update_parameters(slurm_job_name=name)
    executor.submit(run_calc, dfi, DEBUG, N_CORES, MEM_GB)
    print(f"Submitted: {name}")

Submitted: ts34-ts3
Submitted: ts34-ts4
